In [3]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize confi||g with toml file
c = sfp.SolutionFilesConfig('config_files/ukraine/UKR.toml')

# c = sfp.SolutionFilesConfig('config_files/thailand/THA.toml')
from solution_file_processing.plots import create_plot_1a, create_plot_2b_ref_plots, create_plot_2_summary, create_plot_10_ts_by_model
from solution_file_processing.plots import create_plot_6_ldc_and_line_plots, create_plot_7_co2_savings, create_plot_8_services, create_plot_9_av_cap
from solution_file_processing.plots import create_plot_1a_overall_models

[17:22:39 00:22] I:121:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20241129_172239-UKR.log.
[17:22:40 00:00] I:166:__init__ - Initialized SolutionFilesConfig for config_files/ukraine/UKR.toml.


In [12]:
c.v.build_cost_by_plant

PLEXOS technology,Battery,Biomass Power Plant,CHP BiomassandWaste,CHP Coal,CHP Gas,CHP Oil,Coal Subcritical 5,Gas GT 1,Gas GT 3,Hydro PSH,Hydro Reservoir,Hydro RoR,Hydro RoRpondage,Nuclear,Solar PV Buildings,Solar PV Utility,Wind Onshore
model,,,,,,,,,,,,,,,,,
UKR_2025-BAU_LT_eG_Td,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,344764.707900,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
UKR_2025-BAU_LT_eG_Td_CO2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,345283.931857,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
UKR_2025-BAU_LT_eG_Td_Rn,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,648510.723143,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
UKR_2025-BAU_LT_eG_Td_Rn_CO2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,648510.723143,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
UKR_2025-BAU_LT_eSBWaG_Td_CO2,337571.683813,0.0,0.0,0.0,0.0,0.0,0.0,84114.281144,0.0,0.0,0.0,0.0,0.0,0.0,251902.814049,0.0,161582.979733
UKR_2025-BAU_LT_eSBaG_Td,227585.525492,0.0,0.0,0.0,0.0,0.0,0.0,185882.176849,0.0,0.0,0.0,0.0,0.0,0.0,37335.941057,0.0,0.000000
UKR_2025-BAU_LT_eSBaG_Td_CO2,351823.325282,0.0,0.0,0.0,0.0,0.0,0.0,116825.390478,0.0,0.0,0.0,0.0,0.0,0.0,388650.055961,0.0,0.000000
UKR_2025-BAU_LT_eSBaG_Td_Ix,230460.303112,0.0,0.0,0.0,0.0,0.0,0.0,131363.661293,0.0,0.0,0.0,0.0,0.0,0.0,38130.322782,0.0,0.000000
UKR_2025-BAU_LT_eSBaG_Td_Ix_CO2,337899.842248,0.0,0.0,0.0,0.0,0.0,0.0,67499.114498,0.0,0.0,0.0,0.0,0.0,0.0,374545.921781,0.0,0.000000


In [11]:
c.o.gen_yr_df.WEO_Tech_simpl.unique()

array(['Biomass', 'Hydro', 'Solar', 'Wind', 'Bituminous coal', 'Gas-OCGT',
       'Oil', 'Uranium', 'Battery'], dtype=object)

In [ ]:
c.o.gen_yr_df.WEO_tech.unique()

In [ ]:
create_plot_1a_overall_models(c, summary_table=c.v.use_summary, plot_name='use')

In [ ]:
create_plot_1a_overall_models(c, summary_table=c.v.use_summary, plot_name='use')